In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint



In [2]:
import_source="https://raw.githubusercontent.com/SimbarasheJehol/MachineLearningProject/master/dataProccesed2.csv"
df = pd.read_csv(import_source)
print("data is loaded")

data is loaded


In [3]:
df.columns

Index(['Unnamed: 0', 'age', 'workclass', 'fnlwgt', 'education',
       'education_num', 'martial_status', 'occupation', 'RelationShip', 'race',
       'sex', 'capital_gain', 'capital_loss', 'hours_per_week',
       'native_country', 'age-binned', 'fnlwgt-binned', 'education_num-binned',
       'capital_gain-binned', 'capital_loss-binned', 'hours_per_week-binned',
       'education_num_rand', 'income_class'],
      dtype='object')

In [4]:
df=df.drop('Unnamed: 0',axis=1)

In [5]:
df=df.drop( "fnlwgt", axis=1)\
                .drop( "capital_gain",axis=1)\
                .drop("capital_loss",axis=1)\
                .drop("education_num_rand",axis=1)\
                .drop( "fnlwgt-binned", axis=1)\
                .drop( "capital_gain-binned",axis=1)\
                .drop("capital_loss-binned",axis=1)\
                .drop("education_num-binned",axis=1)\
                .drop("hours_per_week-binned",axis=1)\

In [6]:
df.shape

(32561, 13)

In [7]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [8]:
train_df, test_df = train_test_split(df, test_size=0.2)

In [9]:
test_df.sample(5)

,age,workclass,education,education_num,martial_status,occupation,RelationShip,race,sex,hours_per_week,native_country,age-binned,income_class
22460,48,Private,HS-grad,9,Divorced,Sales,Not-in-family,White,Male,60,United-States,senior_workclass,<=50K
13013,51,Self-emp-inc,Assoc-acdm,12,Never-married,Sales,Not-in-family,White,Female,25,United-States,Advanced_workclass,<=50K
17032,36,Federal-gov,Assoc-voc,11,Married-civ-spouse,Tech-support,Husband,White,Male,40,United-States,mid_workclass,<=50K
15534,29,Private,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,99,United-States,junior work_class,<=50K
6838,36,Private,Some-college,10,Separated,Adm-clerical,Other-relative,White,Female,40,United-States,mid_workclass,<=50K


In [10]:
data = train_df.values ##speeding up the code
data[:5]

array([[39, ' State-gov', ' Bachelors', 13, ' Never-married',
        ' Adm-clerical', ' Not-in-family', ' White', ' Male', 40,
        ' United-States', 'mid_workclass', ' <=50K'],
       [50, ' Self-emp-not-inc', ' Bachelors', 13, ' Married-civ-spouse',
        ' Exec-managerial', ' Husband', ' White', ' Male', 13,
        ' United-States', 'Advanced_workclass', ' <=50K'],
       [53, ' Private', ' 11th', 7, ' Married-civ-spouse',
        ' Handlers-cleaners', ' Husband', ' Black', ' Male', 40,
        ' United-States', 'Advanced_workclass', ' <=50K'],
       [28, ' Private', ' Bachelors', 13, ' Married-civ-spouse',
        ' Prof-specialty', ' Wife', ' Black', ' Female', 40, ' Cuba',
        'junior work_class', ' <=50K'],
       [49, ' Private', ' 9th', 5, ' Married-spouse-absent',
        ' Other-service', ' Not-in-family', ' Black', ' Female', 16,
        ' Jamaica', 'senior_workclass', ' <=50K']], dtype=object)

In [11]:
def check_purity(data):
    
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False


In [12]:
check_purity(train_df.values)

False

In [13]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [14]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        type_of_feature = FEATURE_TYPES[column_index]
        if type_of_feature == "continuous":
            potential_splits[column_index] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[column_index].append(potential_split)
        
        # feature is categorical
        # (there need to be at least 2 unique values, otherwise in the
        # split_data function data_below would contain all data points
        # and data_above would be empty)
        elif len(unique_values) > 1:
            potential_splits[column_index] = unique_values
    
    return potential_splits

In [15]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [16]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [17]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [18]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [19]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 20
    for feature in df.columns:
        if feature != "income_class":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [20]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [21]:
tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree)

{'martial_status =  Married-civ-spouse': [{'occupation =  Prof-specialty': [' '
                                                                            '>50K',
                                                                            {'occupation =  Exec-managerial': [' '
                                                                                                               '>50K',
                                                                                                               ' '
                                                                                                               '<=50K']}]},
                                          ' <=50K']}


In [22]:
example = test_df.iloc[:,:]
example

,age,workclass,education,education_num,martial_status,occupation,RelationShip,race,sex,hours_per_week,native_country,age-binned,income_class
5747,56,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,40,United-States,Advanced_workclass,>50K
25798,22,Private,Some-college,10,Never-married,Sales,Own-child,White,Female,35,United-States,starter_workclass,<=50K
3535,18,Private,HS-grad,9,Never-married,Other-service,Other-relative,Black,Male,36,United-States,starter_workclass,<=50K
8028,47,Private,HS-grad,9,Divorced,Other-service,Own-child,White,Female,32,United-States,senior_workclass,<=50K
7310,49,Private,HS-grad,9,Married-civ-spouse,Other-service,Wife,Asian-Pac-Islander,Female,8,Philippines,senior_workclass,<=50K
11444,34,Private,Some-college,10,Married-civ-spouse,Other-service,Husband,White,Male,62,Mexico,mid_workclass,<=50K
32341,74,Self-emp-not-inc,Bachelors,13,Widowed,Craft-repair,Not-in-family,White,Male,8,Germany,sineor_retired,>50K
22629,47,Private,HS-grad,9,Divorced,Tech-support,Not-in-family,White,Female,36,United-States,senior_workclass,<=50K
12642,30,Private,Bachelors,13,Married-civ-spouse,Other-service,Wife,White,Female,40,United-States,junior work_class,<=50K
11234,17,Private,11th,7,Never-married,Prof-specialty,Own-child,White,Female,40,United-States,starter_workclass,<=50K


In [23]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" "),question.split(" "),question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [24]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, args=(tree,), axis=1)
    df["classification_correct"] = df["classification"] == df["income_class"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [25]:
calculate_accuracy(test_df,tree)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:851: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


0.7589066339066339

In [26]:
train_df, test_df = train_test_split(df, test_size=0.3)
tree = decision_tree_algorithm(train_df)
accuracy = calculate_accuracy(test_df, tree)

In [27]:
test_df


,age,workclass,education,education_num,martial_status,occupation,RelationShip,race,sex,hours_per_week,native_country,age-binned,income_class,classification,classification_correct
7822,61,Private,HS-grad,9,Married-civ-spouse,Protective-serv,Husband,White,Male,40,United-States,diminishing_workclass,<=50K,<=50K,True
11152,32,Private,Assoc-acdm,12,Married-civ-spouse,Adm-clerical,Husband,White,Male,45,United-States,junior work_class,>50K,<=50K,False
29877,49,Self-emp-not-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,senior_workclass,<=50K,<=50K,True
5021,44,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,senior_workclass,<=50K,<=50K,True
31918,26,Private,Some-college,10,Never-married,Machine-op-inspct,Not-in-family,Asian-Pac-Islander,Male,45,United-States,junior work_class,>50K,<=50K,False
23639,56,Local-gov,Some-college,10,Married-civ-spouse,Adm-clerical,Wife,White,Female,15,United-States,Advanced_workclass,<=50K,<=50K,True
12680,43,Private,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,40,United-States,senior_workclass,>50K,<=50K,False
28868,29,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Black,Male,55,United-States,junior work_class,>50K,<=50K,False
9812,44,Local-gov,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,40,United-States,senior_workclass,>50K,<=50K,False
16626,36,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,44,United-States,mid_workclass,<=50K,<=50K,True


In [28]:
pprint(tree,width=5)

{'martial_status =  Married-civ-spouse': [{'occupation =  Prof-specialty': [{'education_num = 15': [{'age <= 48.5': [' '
                                                                                                                     '>50K',
                                                                                                                     {'hours_per_week <= 18.0': [' '
                                                                                                                                                 '<=50K',
                                                                                                                                                 ' '
                                                                                                                                                 '>50K']}]},
                                                                                                    {'age <= 32.5': [{'age <= 26.5': [' '
       